# Exercise 3.2

Import the pySpark libraries

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyspark

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==2019.10=py37_0
  - defaults/noarch::dask==2.5.2=py_0
  - defaults/linux-64::statsmodels==0.10.1=py37hdd07704_0
  - conda-forge/noarch::pyspark==2.4.5=py_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::seaborn==0.9.0=py37_0
done

## Package Plan ##

  environment location: /home/bas/anaconda3

  added / updated specs:
    - pyspark


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           6 KB
    anaconda-custom            |           py37_1           3 KB
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py37hc8dfbb8_0         151 KB  conda-forge
    conda-4.8.3              

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, size
spark = SparkSession.builder.appName("Packt").getOrCreate()

Connect to a local Spark session

In [4]:
spark = SparkSession.builder.appName("Packt").getOrCreate()

Read the raw data from a CSV file

In [5]:
data = spark.read.csv('../../Datasets/netflix_titles_nov_2019.csv', header='true')
data.show()

+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+-------+
| show_id|               title|            director|                cast|             country|       date_added|release_year|rating| duration|           listed_in|         description|   type|
+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+-------+
|81193313|           Chocolate|                null|Ha Ji-won, Yoon K...|         South Korea|November 30, 2019|        2019| TV-14| 1 Season|International TV ...|Brought together ...|TV Show|
|81197050|Guatemala: Heart ...|Luis Ara, Ignacio...|   Christian Morales|                null|November 30, 2019|        2019|  TV-G|   67 min|Documentaries, In...|From Sierra de la...|  Movie|
|81213894|     The Zoya Factor|    

Take only the movies

In [7]:
movies = data.filter((col('type') == 'Movie') & (col('release_year') == 2019))
movies.show()

+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+-----+
| show_id|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description| type|
+--------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+-----+
|81197050|Guatemala: Heart ...|Luis Ara, Ignacio...|   Christian Morales|                null|November 30, 2019|        2019|  TV-G|  67 min|Documentaries, In...|From Sierra de la...|Movie|
|81213894|     The Zoya Factor|     Abhishek Sharma|Sonam Kapoor, Dul...|               India|November 30, 2019|        2019| TV-14| 135 min|Comedies, Dramas,...|A goofy copywrite...|Movie|
|81082007|           Atlantics|           Mati Dio

Add a column with the number of actors

In [8]:
transformed = movies.withColumn('count_cast', size(split(movies['cast'], ',')))

Select a subset of columns to store

In [9]:
selected = transformed.select('title', 'director', 'count_cast', 'cast', 'rating', 'release_year', 'type')
selected.show()

+--------------------+--------------------+----------+--------------------+------+------------+-----+
|               title|            director|count_cast|                cast|rating|release_year| type|
+--------------------+--------------------+----------+--------------------+------+------------+-----+
|Guatemala: Heart ...|Luis Ara, Ignacio...|         1|   Christian Morales|  TV-G|        2019|Movie|
|     The Zoya Factor|     Abhishek Sharma|         8|Sonam Kapoor, Dul...| TV-14|        2019|Movie|
|           Atlantics|           Mati Diop|         9|Mama Sane, Amadou...| TV-14|        2019|Movie|
|      I Lost My Body|       Jérémy Clapin|         6|Hakim Faris, Vict...| TV-MA|        2019|Movie|
|        Holiday Rush|        Leslie Small|         9|Romany Malco, Son...| TV-PG|        2019|Movie|
| Evvarikee Cheppoddu|Basava Shankar Eeday|         5|Rakesh Varre, Gar...| TV-14|        2019|Movie|
|Little Singham: M...|       Prakash Satam|         5|Saumya Daan, Sona...| TV-Y7|

Write the contents of the DataFrame to disk

In [11]:
selected.write.csv('transformed', header='true')

In [12]:
# note: the actual name of the csv file ('part-....') differs on each run
!head transformed/part-00000-873224d4-6aa0-4df8-bb63-2d75aae9e706-c000.csv

title,director,count_cast,cast,rating,release_year,type
Guatemala: Heart of the Mayan World,"Luis Ara, Ignacio Jaunsolo",1,Christian Morales,TV-G,2019,Movie
The Zoya Factor,Abhishek Sharma,8,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, Sikander Kher, Angad Bedi, Koel Purie, Pooja Bhamrah, Manu Rishi Chadha",TV-14,2019,Movie
Atlantics,Mati Diop,9,"Mama Sane, Amadou Mbow, Ibrahima Traore, Nicole Sougou, Amina Kane, Mariama Gassama, Coumba Dieng, Ibrahima Mbaye, Diankou Sembene",TV-14,2019,Movie
I Lost My Body,Jérémy Clapin,6,"Hakim Faris, Victoire Du Bois, Patrick d'Assumçao, Dev Patel, Alia Shawkat, George Wendt",TV-MA,2019,Movie
Holiday Rush,Leslie Small,9,"Romany Malco, Sonequa Martin-Green, Darlene Love, Deon Cole, La La Anthony, Deysha Nelson, Amarr M. Wooten, Selena-Marie Alphonse, Andrea-Marie Alphonse",TV-PG,2019,Movie
Evvarikee Cheppoddu,Basava Shankar Eeday,5,"Rakesh Varre, Gargeyi, Vamsi raj Nekkanti, D P Ghani, K Prasanna",TV-14,2019,Movie
Little Singham: Mahabali,Prakash S